<a href="https://colab.research.google.com/github/HJapok/Palmoil_HDE_Estimate/blob/master/Harvest_Estimation_InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
import os
import pandas as pd
  
# Get the list of all files and directories
# in the root directory
path = "/content/drive/MyDrive/synthetic_dataset"
dir_list = os.listdir(path)
data_arr = []
for i in dir_list:
    img_fol=os.path.join(path, str(i))
    img_list = os.listdir(img_fol)
    for k in img_list:
        data={
            'Filepath':os.path.join(img_fol, str(k)),
            'Day':int(str(k).split('_')[2])
                }
        data_arr.append(data)


image_df = pd.DataFrame(data_arr, 
             columns=['Filepath', 
                      'Day'])
print(image_df.head(5))
        

                                            Filepath  Day
0  /content/drive/MyDrive/synthetic_dataset/ripe/...  130
1  /content/drive/MyDrive/synthetic_dataset/ripe/...  130
2  /content/drive/MyDrive/synthetic_dataset/ripe/...  130
3  /content/drive/MyDrive/synthetic_dataset/ripe/...  130
4  /content/drive/MyDrive/synthetic_dataset/ripe/...  131


In [ ]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_df

,Filepath,Day
5838,/content/drive/MyDrive/synthetic_dataset/unrip...,17
9406,/content/drive/MyDrive/synthetic_dataset/overr...,141
15007,/content/drive/MyDrive/synthetic_dataset/under...,58
280,/content/drive/MyDrive/synthetic_dataset/ripe/...,134
5625,/content/drive/MyDrive/synthetic_dataset/unrip...,35
...,...,...
905,/content/drive/MyDrive/synthetic_dataset/ripe/...,140
5192,/content/drive/MyDrive/synthetic_dataset/unrip...,31
12172,/content/drive/MyDrive/synthetic_dataset/under...,87
235,/content/drive/MyDrive/synthetic_dataset/ripe/...,133


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Day',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=8,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Day',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=8,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Day',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=8,
    shuffle=False
)

Found 8624 validated image filenames.
Found 2156 validated image filenames.
Found 4620 validated image filenames.


In [ ]:
N_mobile = tf.keras.applications.inception_v3.InceptionV3( input_tensor = tf.keras.Input(
    shape=(224, 224, 3)), 
    include_top=False, 
    weights='imagenet'
    )

N_mobile.trainable = True
base_model_output = N_mobile.output
flattened_output = tf.keras.layers.GlobalAveragePooling2D()(base_model_output)
class_prediction = tf.keras.layers.Dense(64, activation="relu")(flattened_output)
class_prediction = tf.keras.layers.Dense(64, activation="relu")(class_prediction)
class_prediction = tf.keras.layers.Dense(1, activation='linear')(class_prediction)
model = tf.keras.models.Model(inputs=N_mobile.input, outputs=  class_prediction)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200),
    loss='mse',
    metrics=['mse', 'mae']
)
model.summary()

87924736/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 batch_normalization_17 (BatchN  (None, 25, 25, 96)  288         ['conv2d_17[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_18 (BatchN  (None, 25, 25, 64)  192         ['conv2d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_12 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_12[0][0]'] 
                                                                                                  
 activation_14 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_14[0][0]'] 
                                                                                                  
 activatio

In [ ]:
import os
checkpoint_filepath = '/content/checkpoint'
if not os.path.exists(checkpoint_filepath):
   os.makedirs(checkpoint_filepath)

filepath=os.path.join(checkpoint_filepath,"weights-improvement-{epoch:02d}-{val_mse:.2f}.hdf5")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    monitor='val_mse',
    mode='min',
    save_best_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_mse',
            patience=100,
            restore_best_weights=True
        )

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    verbose=1,
    callbacks=[model_checkpoint_callback,early_stopping]
)

Epoch 1/100
270/270 [==============================] - 3843s 14s/step - loss: 312.8863 - mse: 312.8863 - mae: 10.4269 - val_loss: 1356.6105 - val_mse: 1356.6105 - val_mae: 31.7619
Epoch 2/100
270/270 [==============================] - 73s 270ms/step - loss: 35.3973 - mse: 35.3973 - mae: 4.4829 - val_loss: 73.6997 - val_mse: 73.6997 - val_mae: 7.1222
Epoch 3/100
270/270 [==============================] - 75s 279ms/step - loss: 9.7681 - mse: 9.7681 - mae: 2.4272 - val_loss: 21.9361 - val_mse: 21.9361 - val_mae: 3.4231
Epoch 4/100
270/270 [==============================] - 75s 278ms/step - loss: 9.5043 - mse: 9.5043 - mae: 2.3157 - val_loss: 306.0635 - val_mse: 306.0635 - val_mae: 11.6855
Epoch 5/100
270/270 [==============================] - 75s 279ms/step - loss: 6.6147 - mse: 6.6147 - mae: 1.9834 - val_loss: 2.7784 - val_mse: 2.7784 - val_mae: 1.3040
Epoch 6/100
270/270 [==============================] - 75s 278ms/step - loss: 6.5785 - mse: 6.5785 - mae: 1.9323 - val_loss: 71.1058 - va

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import os
model_filepath = '/content/drive/MyDrive/Model_InceptionV3'
if not os.path.exists(model_filepath):
   os.makedirs(model_filepath)

# Save model in json and model weight by h5 
model_json = model.to_json()
with open(os.path.join(model_filepath,"model.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(model_filepath,"model.h5"))
print("Saved model to disk")

Saved model to disk


In [ ]:
# load new model in json format
model_filepath = '/content/drive/MyDrive/Model_InceptionV3'
json_file = open(os.path.join(model_filepath,"model.json"), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/Model_InceptionV3/weights-improvement-05-0.17.hdf5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
loaded_model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200),
    loss='mse',
    metrics=['mse', 'mae']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Retraining
filepath=os.path.join(checkpoint_filepath,"weights-improvement-{epoch:02d}-{val_mse:.2f}.hdf5")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    monitor='val_mse',
    mode='min',
    save_best_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_mse',
            patience=100,
            restore_best_weights=True
        )

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=25,
    verbose=1,
    callbacks=[model_checkpoint_callback,early_stopping]
)

Epoch 1/50
270/270 [==============================] - 6879s 25s/step - loss: 0.7732 - mse: 0.7732 - mae: 0.5823 - val_loss: 6.4561 - val_mse: 6.4561 - val_mae: 2.1049
Epoch 2/50
270/270 [==============================] - 76s 283ms/step - loss: 0.5648 - mse: 0.5648 - mae: 0.5602 - val_loss: 24.2968 - val_mse: 24.2968 - val_mae: 4.4357
Epoch 3/50
270/270 [==============================] - 76s 280ms/step - loss: 1.0121 - mse: 1.0121 - mae: 0.6954 - val_loss: 0.3846 - val_mse: 0.3846 - val_mae: 0.3942
Epoch 4/50
270/270 [==============================] - 76s 280ms/step - loss: 1.1064 - mse: 1.1064 - mae: 0.8040 - val_loss: 55.7409 - val_mse: 55.7409 - val_mae: 7.1198
Epoch 5/50
270/270 [==============================] - 76s 281ms/step - loss: 0.5261 - mse: 0.5261 - mae: 0.5538 - val_loss: 10.3529 - val_mse: 10.3529 - val_mae: 2.7455
Epoch 6/50
270/270 [==============================] - 76s 280ms/step - loss: 0.5549 - mse: 0.5549 - mae: 0.5762 - val_loss: 46.4662 - val_mse: 46.4662 - val_ma

In [ ]:
# Retrain and resave
# Save model in json and model weight by h5 
model_json = loaded_model.to_json()
with open("/content/drive/MyDrive/model_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
loaded_model.save_weights("/content/drive/MyDrive/model_weight_2.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# Direct save model together with model and ist weight
loaded_model.save('/content/drive/MyDrive/model_2.h5')

In [ ]:
# Load the model
modeling = tf.keras.models.load_model('/content/drive/MyDrive/model_2.h5')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
predicted_ages = np.squeeze(loaded_model.predict(test_images))
true_ages = test_images.labels

rmse = mean_squared_error(true_ages, predicted_ages)
print("rmse: ", rmse)

r2 = r2_score(true_ages, predicted_ages)
print("r2: ",r2)

rmse:  1.5405133481143352
r2:  0.9992103341625127


In [ ]:
from PIL import Image
image_path = "/content/drive/MyDrive/synthetic_dataset/unripe/unripe_day_45_pic_32.png"
image = tf.keras.preprocessing.image.load_img(image_path)
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])
input_arr*1./255
input_arr=input_arr*1./255
input_arr
predictions = modeling.predict(input_arr)
print(np.squeeze(predictions))

45.453056
